In [ ]:
import torch.nn as nn
import torch

from torch.onnx import register_custom_op_symbolic, symbolic_helper


def router_op(g: torch._C.Graph, n: torch._C.Node, *args, **kwargs):
    print(kwargs["name"])
    return g.op("brt::Router", args[0])

register_custom_op_symbolic("::prim_PythonOp", router_op, 1)


class Router(nn.Module):
    @torch.jit.ignore
    def forward(self, in_data):
        return in_data


class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.router = Router()

    def forward(self, in_data):
        return self.router(in_data)


router = Model()

script_router = torch.jit.script(router)

for node in script_router.graph.nodes():
    print(
        node, node.kind(),
    )
print(script_router.graph)

input = torch.Tensor(1, 1, 1, 1)
torch.onnx.export(script_router, input, "test.onnx")


In [ ]:
import torchvision

DeformConv = torchvision.ops.DeformConv2d